In [2]:
import pandas as pd
import os, sys

# Filter all positive images out

In [3]:
# Get all positive #1 (活动性出血) images -- 429 total images

In [4]:
import os, sys, glob
pos_ind_df = pd.read_excel(r'C:\Users\wiqwe\Desktop\上交科研\c活动性出血.xlsx')
pos_ind_df.columns
name_ind = pos_ind_df[[' 姓名','出血典型病变图片号码']]

In [85]:
# traverse all positive image folders
import re
folder_prefix = r'C:\Users\wiqwe\Desktop\上交科研\阳性'
folders = glob.glob(os.path.join(folder_prefix,'*'))
# print(folders)
name_to_folder = {}

def name_extract(folder):
    pattern = re.compile('\d{6,10}(.+)')
    name = pattern.findall(folder)
    name_to_folder[name[0]] = folder
    return name

names = [name_extract(i) for i in folders]

In [ ]:
# Copy selected images from patient's folder to dataset folder
import shutil
pos_img_index = 0
POSITIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_positive_#1_imgs'
def func(a):
    name, img_inds = a.tolist()
    if name.endswith('2'): name = name[:-1]
#     print(name, img_inds,'|', type(img_inds), len(img_inds), 'called')
    img_inds = img_inds.replace(' ', '').replace('，', ',').strip(',').replace('.',',').split(',')
    name = name.strip()
#     print(type(img_inds), img_inds)
    
    def copy_file_func(img_ind):
        global pos_img_index
        full_path = os.path.join(name_to_folder[name], img_ind + '.jpg')
        print(full_path)
        paste_path = os.path.join(POSITIVE_IMGS_FOLDER, str(pos_img_index) + '.jpg')
        shutil.copyfile(full_path,paste_path)
        pos_img_index += 1
   
    _ = [copy_file_func(i) for i in img_inds]
#     print(name, img_inds)
    
_ = name_ind.apply(func, axis=1, raw=True)

In [5]:
# Get all positive #2 (血管畸形) images -- 429 total images
pos_ind_df = pd.read_excel(r'C:\Users\wiqwe\Desktop\上交科研\消化道出血病人表-血管畸形80.xlsx')
pos_ind_df.columns
# name_ind = pos_ind_df[[' 姓名','病变图片编号']]

Index([' 姓名', ' 检查日期', '工程师加密', ' 内镜诊断', 'Unnamed: 4', '病变图片编号', '其他'], dtype='object')

In [101]:
# Get negative images -- 454
import random
NEGATIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs'
folder_prefix = r'C:\Users\wiqwe\Desktop\上交科研\阴性'
neg_img_index = 0
folders = glob.glob(os.path.join(folder_prefix,'*'))
for folder in folders:
    images = glob.glob(os.path.join(folder, '*'))
    if not images: continue
    image = random.choice(images)
    image_basename = os.path.basename(image)
    paste_path = os.path.join(NEGATIVE_IMGS_FOLDER, str(neg_img_index) + '.jpg')
    neg_img_index += 1
#     print(image)
    shutil.copyfile(image,paste_path)


In [1]:
import tensorflow as tf
import keras
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
keras.backend.set_session(sess)

Using TensorFlow backend.


In [55]:
# image preprocessing pipeline (add positive and negative images)
import numpy as np
import PIL.Image
import glob, os
BLEEDING_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_bleeding_imgs'
NEGATIVE_IMGS_FOLDER = r'C:\Users\wiqwe\Desktop\上交科研\selected_negative_imgs'

train_data = []
train_label = []
validation_data = []
validation_label = []
for img in glob.glob(os.path.join(POSITIVE_IMGS_FOLDER ,'*')):
    im = PIL.Image.open(img)
    im = im.resize((224,224))
    im = np.array(im)
    im = (im / 255.0) - 1.0
#     im_roll = np.rollaxis(im, 2, 0)
#     assert (im[:,:,0] == im_roll[0,:,:]).all()
#     assert (im[:,:,1] == im_roll[1,:,:]).all()
#     assert (im[:,:,2] == im_roll[2,:,:]).all()
    train_data.append(np.array(im))
print(len(train_data), train_data[0].shape)
train_label += [[0, 1] for i in range(len(train_data))]

for img in glob.glob(os.path.join(NEGATIVE_IMGS_FOLDER ,'*')):
    im = PIL.Image.open(img)
    im = im.resize((224,224))
    im = np.array(im)
    im = (im / 255.0) - 1.0
#     im_roll = np.rollaxis(im, 2, 0)
#     assert (im[:,:,0] == im_roll[0,:,:]).all()
#     assert (im[:,:,1] == im_roll[1,:,:]).all()
#     assert (im[:,:,2] == im_roll[2,:,:]).all()
    train_data.append(np.array(im))
print(len(train_data), train_data[0].shape)
train_label += [[1, 0] for i in range(len(train_data) - len(train_label))]

429 (224, 224, 3)
883 (224, 224, 3)


In [49]:
recovered_img = ((train_data[0]+1)*255).astype(np.uint8)
print(recovered_img.shape)
PIL.Image.fromarray(recovered_img).show()

(224, 224, 3)


In [ ]:
# split the data to be train and validation set
import random
VALIDATION_RATIO = 0.1
num_validation = int(len(train_data) * VALIDATION_RATIO)
validation_indices = random.sample(list(range(len(train_data))))
validation_data = train_data[validation_indices]
validation_label = train_label[validation_indices]


In [ ]:
train_data = np.stack(train_data, axis=0)
train_label = np.array(train_label)
print(train_data.shape, train_label.shape)

In [58]:
resnet_50 = ResnetBuilder.build_resnet_50((3,224,224), 2)

In [59]:
optimizer = keras.optimizers.Adam(lr=0.001, amsgrad=True, decay=0.00001)
resnet_50.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['categorical_accuracy'])

In [60]:
resnet_50.fit(train_data, train_label, epochs=10, batch_size=32, validation_split = 0.1)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 794 samples, validate on 89 samples
Epoch 1/10
794/794 [==============================] - 146s 184ms/step - loss: 5.6880 - categorical_accuracy: 0.7557 - val_loss: 9.3965 - val_categorical_accuracy: 0.0674
Epoch 2/10
794/794 [==============================] - 16s 21ms/step - loss: 4.7413 - categorical_accuracy: 0.8904 - val_loss: 20.2437 - val_categorical_accuracy: 0.0000e+00
Epoch 3/10
794/794 [==============================] - 16s 20ms/step - loss: 3.9602 - categorical_accuracy: 0.9610 - val_loss: 12.5301 - val_categorical_accuracy: 0.0112
Epoch 4/10
794/794 [==============================] - 16s 20ms/step - loss: 3.4109 - categorical_accuracy: 0.9597 - val_loss: 12.9912 - val_categorical_accuracy: 0.0000e+00
Epoch 5/10
794/794 [==============================] - 16s 20ms/step - loss: 2.9604 - categorical_accuracy: 0.9572 - val_loss: 7.4522 - val_categorical_accuracy: 0.0787
Epoch 6/

# keras resnet definition

In [40]:
from __future__ import division

import six

from keras.models import Model

from keras.layers import (

    Input,

    Activation,

    Dense,

    Flatten

)

from keras.layers.convolutional import (

    Conv2D,

    MaxPooling2D,

    AveragePooling2D

)

from keras.layers.merge import add

from keras.layers.normalization import BatchNormalization

from keras.regularizers import l2

from keras import backend as K





def _bn_relu(input):

    """Helper to build a BN -> relu block

    """

    norm = BatchNormalization(axis=CHANNEL_AXIS)(input)

    return Activation("relu")(norm)





def _conv_bn_relu(**conv_params):

    """Helper to build a conv -> BN -> relu block

    """

    filters = conv_params["filters"]

    kernel_size = conv_params["kernel_size"]

    strides = conv_params.setdefault("strides", (1, 1))

    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

    padding = conv_params.setdefault("padding", "same")

    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))



    def f(input):

        conv = Conv2D(filters=filters, kernel_size=kernel_size,

                      strides=strides, padding=padding,

                      kernel_initializer=kernel_initializer,

                      kernel_regularizer=kernel_regularizer)(input)

        return _bn_relu(conv)



    return f





def _bn_relu_conv(**conv_params):

    """Helper to build a BN -> relu -> conv block.

    This is an improved scheme proposed in http://arxiv.org/pdf/1603.05027v2.pdf

    """

    filters = conv_params["filters"]

    kernel_size = conv_params["kernel_size"]

    strides = conv_params.setdefault("strides", (1, 1))

    kernel_initializer = conv_params.setdefault("kernel_initializer", "he_normal")

    padding = conv_params.setdefault("padding", "same")

    kernel_regularizer = conv_params.setdefault("kernel_regularizer", l2(1.e-4))



    def f(input):

        activation = _bn_relu(input)

        return Conv2D(filters=filters, kernel_size=kernel_size,

                      strides=strides, padding=padding,

                      kernel_initializer=kernel_initializer,

                      kernel_regularizer=kernel_regularizer)(activation)



    return f





def _shortcut(input, residual):

    """Adds a shortcut between input and residual block and merges them with "sum"

    """

    # Expand channels of shortcut to match residual.

    # Stride appropriately to match residual (width, height)

    # Should be int if network architecture is correctly configured.

    input_shape = K.int_shape(input)

    residual_shape = K.int_shape(residual)

    stride_width = int(round(input_shape[ROW_AXIS] / residual_shape[ROW_AXIS]))

    stride_height = int(round(input_shape[COL_AXIS] / residual_shape[COL_AXIS]))

    equal_channels = input_shape[CHANNEL_AXIS] == residual_shape[CHANNEL_AXIS]



    shortcut = input

    # 1 X 1 conv if shape is different. Else identity.

    if stride_width > 1 or stride_height > 1 or not equal_channels:

        shortcut = Conv2D(filters=residual_shape[CHANNEL_AXIS],

                          kernel_size=(1, 1),

                          strides=(stride_width, stride_height),

                          padding="valid",

                          kernel_initializer="he_normal",

                          kernel_regularizer=l2(0.0001))(input)



    return add([shortcut, residual])





def _residual_block(block_function, filters, repetitions, is_first_layer=False):

    """Builds a residual block with repeating bottleneck blocks.

    """

    def f(input):

        for i in range(repetitions):

            init_strides = (1, 1)

            if i == 0 and not is_first_layer:

                init_strides = (2, 2)

            input = block_function(filters=filters, init_strides=init_strides,

                                   is_first_block_of_first_layer=(is_first_layer and i == 0))(input)

        return input



    return f





def basic_block(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """Basic 3 X 3 convolution blocks for use on resnets with layers <= 34.

    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf

    """

    def f(input):



        if is_first_block_of_first_layer:

            # don't repeat bn->relu since we just did bn->relu->maxpool

            conv1 = Conv2D(filters=filters, kernel_size=(3, 3),

                           strides=init_strides,

                           padding="same",

                           kernel_initializer="he_normal",

                           kernel_regularizer=l2(1e-4))(input)

        else:

            conv1 = _bn_relu_conv(filters=filters, kernel_size=(3, 3),

                                  strides=init_strides)(input)



        residual = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv1)

        return _shortcut(input, residual)



    return f





def bottleneck(filters, init_strides=(1, 1), is_first_block_of_first_layer=False):

    """Bottleneck architecture for > 34 layer resnet.

    Follows improved proposed scheme in http://arxiv.org/pdf/1603.05027v2.pdf



    Returns:

        A final conv layer of filters * 4

    """

    def f(input):



        if is_first_block_of_first_layer:

            # don't repeat bn->relu since we just did bn->relu->maxpool

            conv_1_1 = Conv2D(filters=filters, kernel_size=(1, 1),

                              strides=init_strides,

                              padding="same",

                              kernel_initializer="he_normal",

                              kernel_regularizer=l2(1e-4))(input)

        else:

            conv_1_1 = _bn_relu_conv(filters=filters, kernel_size=(1, 1),

                                     strides=init_strides)(input)



        conv_3_3 = _bn_relu_conv(filters=filters, kernel_size=(3, 3))(conv_1_1)

        residual = _bn_relu_conv(filters=filters * 4, kernel_size=(1, 1))(conv_3_3)

        return _shortcut(input, residual)



    return f





def _handle_dim_ordering():

    global ROW_AXIS

    global COL_AXIS

    global CHANNEL_AXIS

    if K.image_dim_ordering() == 'tf':

        ROW_AXIS = 1

        COL_AXIS = 2

        CHANNEL_AXIS = 3

    else:

        CHANNEL_AXIS = 1

        ROW_AXIS = 2

        COL_AXIS = 3





def _get_block(identifier):

    if isinstance(identifier, six.string_types):

        res = globals().get(identifier)

        if not res:

            raise ValueError('Invalid {}'.format(identifier))

        return res

    return identifier





class ResnetBuilder(object):

    @staticmethod

    def build(input_shape, num_outputs, block_fn, repetitions):

        """Builds a custom ResNet like architecture.



        Args:

            input_shape: The input shape in the form (nb_channels, nb_rows, nb_cols)

            num_outputs: The number of outputs at final softmax layer

            block_fn: The block function to use. This is either `basic_block` or `bottleneck`.

                The original paper used basic_block for layers < 50

            repetitions: Number of repetitions of various block units.

                At each block unit, the number of filters are doubled and the input size is halved



        Returns:

            The keras `Model`.

        """

        _handle_dim_ordering()

        if len(input_shape) != 3:

            raise Exception("Input shape should be a tuple (nb_channels, nb_rows, nb_cols)")



        # Permute dimension order if necessary

        if K.image_dim_ordering() == 'tf':

            input_shape = (input_shape[1], input_shape[2], input_shape[0])



        # Load function from str if needed.

        block_fn = _get_block(block_fn)



        input = Input(shape=input_shape)

        conv1 = _conv_bn_relu(filters=64, kernel_size=(7, 7), strides=(2, 2))(input)

        pool1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding="same")(conv1)



        block = pool1

        filters = 64

        for i, r in enumerate(repetitions):

            block = _residual_block(block_fn, filters=filters, repetitions=r, is_first_layer=(i == 0))(block)

            filters *= 2



        # Last activation

        block = _bn_relu(block)



        # Classifier block

        block_shape = K.int_shape(block)

        pool2 = AveragePooling2D(pool_size=(block_shape[ROW_AXIS], block_shape[COL_AXIS]),

                                 strides=(1, 1))(block)

        flatten1 = Flatten()(pool2)

        dense = Dense(units=num_outputs, kernel_initializer="he_normal",

                      activation="softmax")(flatten1)



        model = Model(inputs=input, outputs=dense)

        return model



    @staticmethod

    def build_resnet_18(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [2, 2, 2, 2])



    @staticmethod

    def build_resnet_34(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, basic_block, [3, 4, 6, 3])



    @staticmethod

    def build_resnet_50(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 6, 3])



    @staticmethod

    def build_resnet_101(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 4, 23, 3])



    @staticmethod

    def build_resnet_152(input_shape, num_outputs):

        return ResnetBuilder.build(input_shape, num_outputs, bottleneck, [3, 8, 36, 3])